# Bash Workspace

## Intro

List all the shells that are currently installed on our machine by reading the `/etc/shell` file:

In [3]:
cat /etc/shells

# /etc/shells: valid login shells
/bin/sh
/usr/bin/sh
/bin/bash
/usr/bin/bash
/bin/rbash
/usr/bin/rbash
/bin/dash
/usr/bin/dash


## A Jupyter kernel for bash

- [github repo](https://github.com/takluyver/bash_kernel)

```sh
cat dog.png | display
echo "<b>Dog</b>, not a cat." | displayHTML
echo "alert('It is known khaleesi\!');" | displayJS
```

In [3]:
ls

README.md		python-workspace.ipynb	run-docker-notebook-d.sh
bash-workspace.ipynb	run-docker-bash.sh	run-docker.sh
golang-workspace.ipynb	run-docker-d.sh


## Handle Arguments And Options

Sometimes, we want to add arguments to provide to our script variables to make it more generic.

- `$1` is the first argument
- `$#` variable returns the number of arguments provided to the command line
- `${#args[@]}` returns the number of args array elements

The `shift` command is another handy tool for managing arguments in bash scripts. It ‘shifts’ all arguments to the left. For instance, if you use shift, `$2` becomes `$1`, `$3` becomes `$2`, and so on. The original first argument is discarded.

### One argument

In [30]:
# ONE ARGUMENT
oneArgument () {
  echo $1
}
echo -e "\nONE ARGUMENT"
oneArgument hello


ONE ARGUMENT
hello


### Multiple Arguments

In [31]:
# MULTIPLE ARGUMENTS
multipleArguments () {
  local args=("$@")
  if [ ${#args[@]} -ne 2 ]; then
    echo "Error: Missing or too many arguments."
    exit 1
  fi
  local helloMessage=(${args[0]})
  local name=(${args[1]})
  echo $helloMessage $name
}
echo -e "\nMULTIPLE ARGUMENTS"
multipleArguments hi John


MULTIPLE ARGUMENTS
hi John


### Simple Option

In [32]:
# SIMPLE OPTION
simpleOption () {
  local args
  local short_presentation=false
  while [ $# -gt 0 ]; do
    case "$1" in
      -s|--short-presentation)
        short_presentation=true
        shift
        ;;
      -*|--*)
        echo "Error: Invalid option: $1"
        ;;
      *)
        args+=("$1")
        shift
        ;;
    esac
  done
  if [ ${#args[@]} -ne 2 ]; then
    echo "Error: Missing or too many arguments."
    exit 1
  fi
  local helloMessage=(${args[0]})
  local name=(${args[1]})
  if [ "$short_presentation" = true ]; then
    echo $helloMessage
  else
    echo $helloMessage $name
  fi
}
echo -e "\nSIMPLE OPTION"
simpleOption hello Anna
simpleOption -s hello Anna


SIMPLE OPTION
hello Anna
hello


### Value Option

In [33]:
# VALUE OPTION
valueOption () {
  local args
  local short_presentation=false
  local allowed_colors="green yellow red"
  check_option() {
    for option in $2; do
      if [ "$option" = "$1" ]; then
        return 0
      fi
    done
    echo "Error: Invalid option value : $1"
    exit 1
  }
  echo_color () {
    case $1 in
    green)
      echo "\033[0;32m${@:2}\033[0m"
      ;;
    yellow)
      echo "\033[0;33m${@:2}\033[0m"
      ;;
    red)
      echo "\033[0;31m${@:2}\033[0m"
      ;;
    *)
      echo "${@:2}"
      ;;
    esac
  }
  while [ $# -gt 0 ]; do
    case "$1" in
      -s|--short-presentation)
        short_presentation=true
        shift
        ;;
      -c=*|--color=*)
        check_option "${1#*=}" "$allowed_colors"
        color="${1#*=}"
        shift
        ;;
      -*|--*)
        echo "Error: Invalid option: $1"
        exit 1
        ;;
      *)
        args+=("$1")
        shift
        ;;
    esac
  done
  if [ ${#args[@]} -ne 2 ]; then
    echo "Error: Missing or too many arguments."
  fi
  local helloMessage=(${args[0]})
  local name=(${args[1]})
  if [ "$short_presentation" = true ]; then
    echo_color $color $helloMessage
  else
    echo_color $color $helloMessage $name
  fi
}

echo -e "\nVALUE OPTION"
valueOption "hi" "Jhon" --color=red
valueOption "what's up" "John" --color=green
valueOption hello Anna --color=yellow


VALUE OPTION
\033hi Jhon\033
\033what's John\033
\033hello Anna\033


### Usage

In [2]:
usage() {
  echo    
  echo "Usage: $0 [...options] <hello_message> <name>"
  echo "Options:"
  echo "  -h, --help                 Show this help message."
  echo "  -c, --color COLOR          Set the color for the message (green, yellow or red)."
  echo "  -s, --short-presentation   Display presentation message without name."
  echo
  echo "Example: $0 -c green 'Hello' 'John'"
  echo
}
usage


Usage: /usr/bin/bash [...options] <hello_message> <name>
Options:
  -h, --help                 Show this help message.
  -c, --color COLOR          Set the color for the message (green, yellow or red).
  -s, --short-presentation   Display presentation message without name.

Example: /usr/bin/bash -c green 'Hello' 'John'



In [14]:
# SIMPLE USAGE
simpleUsage () {
  local args
  local short_presentation=false
  local allowed_colors="green yellow red"
  usage() {
    echo
    echo "Usage: $0 [-s|--short-presentation] [-c|--color=<green|yellow|red>] [-h|--help] <hello_message> <name>"
    echo
  }
  check_option() {
    for option in $2; do
      if [ "$option" = "$1" ]; then
        return 0
      fi
    done
    echo "Error: Invalid option value : $1"
    usage
    exit 1
  }
  echo_color () {
    case $1 in
    green)
      echo "\033[0;32m${@:2}\033[0m"
      ;;
    yellow)
      echo "\033[0;33m${@:2}\033[0m"
      ;;
    red)
      echo "\033[0;31m${@:2}\033[0m"
      ;;
    *)
      echo "${@:2}"
      ;;
    esac
  }
  while [ $# -gt 0 ]; do
    case "$1" in
      -h|--help)
        usage
        exit 1
        ;;
      -s|--short-presentation)
        short_presentation=true
        shift
        ;;
      -c=*|--color=*)
        check_option "${1#*=}" "$allowed_colors"
        color="${1#*=}"
        shift
        ;;
      -*|--*)
        echo "Error: Invalid option: $1"
        usage
        exit 1
        ;;
      *)
        args+=("$1")
        shift
        ;;
    esac
  done
  if [ ${#args[@]} -ne 2 ]; then
    echo "Error: Missing or too many arguments."
    usage
    exit 1
  fi
  local helloMessage=(${args[0]})
  local name=(${args[1]})
  if [ "$short_presentation" = true ]; then
    echo_color $color $helloMessage
  else
    echo_color $color $helloMessage $name
  fi
}

echo -e "\nSIMPLE USAGE"
# simpleUsage -h
# simpleUsage "what's up" "John" "how's it going"
# simpleUsage hello Anna --color=purple
simpleUsage hello Anna -d


SIMPLE USAGE
Error: Invalid option: -d

Usage: /usr/bin/bash [-s|--short-presentation] [-c|--color=<green|yellow|red>] [-h|--help] <hello_message> <name>

exit
Restarting Bash
